In [ ]:
!pip install qiskit qiskit_aer
!pip install numpy matplotlib scipy pylatexenc



In [3]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from numpy import pi
import math

ModuleNotFoundError: No module named 'qiskit'

# Ramsey interferometry with the mathmatical foundation

Initializing a qubit in the state $ |\alpha\rangle  = a|0\rangle + b|1\rangle $

Applying a phase shift of magnitude $\frac{π}{2}$

$ |\alpha\rangle  = a|0\rangle + be^{\frac{2i}{\pi}}|1\rangle $

In [ ]:
# create a quantum register with 1 qubit and 1 classical bit
numOfQubits = 1
numOfClassBits = 1
circuit = QuantumCircuit(numOfQubits,numOfClassBits)

# reset the qubit to the 0 state
circuit.reset(0)

# Phase Shift: perform a 90 degree rotation around the K axis
phaseAmount = pi/2
circuit.u(0, phaseAmount, 0, 0)
# measure the qubit and provide a histogram of the results
circuit.measure(0,0)

# print the circuit
circuit.draw('mpl')

Measuring the qubit at this stage will yield the same probabilities as before the phase shift. Consequently, the phase shift is not reflected in the readout, making it impossible to determine the phase.
    

$P[|0\rangle] = a^2 $


$P[|1\rangle] =  | be^{\frac{2i}{\pi}}|^2 =  be^{\frac{2i}{\pi}} *  be^{\frac{-2i}{\pi}} = b^2$

In our example, since we started with the qubit initialized in the 0 state, we should expect a readout of 0 when we measure the circuit, as the phase shift was not encoded.

In [ ]:
# transpile the circuit to the Aer simulator
simulator = Aer.get_backend('qasm_simulator')
compiled_circuit = transpile(circuit, simulator)

# Number of times the experiment is run
shots = 1000

# plot a histogram shots is the number of times the circuit is run(will be elaborated on later)
job = simulator.run(compiled_circuit, shots=shots)
result = job.result()

counts = result.get_counts()

# probbility of the qubit being in the 0 state
probability0 = 0 if '0' not in counts else counts['0']/shots
probability1 = 0 if '1' not in counts else counts['1']/shots

print("Probability of the qubit being in the 0 state: ", probability0)
print("Probability of the qubit being in the 1 state: ", probability1)

# Ramsey Experiment Setup

To encode the phase shift onto the readout probabilities, we will first apply a Hadamard gate to the qubit, rotating it to the state
$$\ket{\alpha} = \frac{\ket{1} + \ket{0}}{\sqrt{2}}$$

In [ ]:

# create a quantum register with 1 qubit and 1 classical bit
numOfQubits = 1
numOfClassBits = 1
circuit = QuantumCircuit(numOfQubits,numOfClassBits)

# reset the qubit to the 0 state
circuit.reset(0)

# ramsey sequence 1: hadamard gate to rotate the qubit to the superposition state
circuit.h(0)




The qubit will then undergo a phase change of magnitude $\theta$ (for our test, its magnitude is $\frac{\pi}{2}$) and end up in the state

$$
\ket{\alpha} = \frac{a\ket{0}}{\sqrt{2}} + \frac{e^{i\theta} b\ket{1}}{\sqrt{2}}
$$

In [ ]:

# Phase Shift: perform a 90 degree rotation around the K axis
phaseAmount = pi/2
circuit.u(0, phaseAmount, 0, 0)



A second Hadamard gate is applied to map the phase shift onto the probability of the qubit being in the 0 or 1 state.

$$
\ket{\alpha} = H \otimes \left( \frac{a\ket{0}}{\sqrt{2}} + \frac{e^{i\theta} b\ket{1}}{\sqrt{2}} \right)
$$

$$
\ket{\alpha} = \frac{1+e^{i\theta}}{2}\ket{0} + \frac{1-e^{i\theta}}{2}\ket{1}
$$

In [ ]:
# ramsey sequence 2: hadamard gate to rotate the qubit back to the 0 or 1 state
circuit.h(0)

# measure the qubit and provide a histogram of the results
circuit.measure(0,0)

# print the circuit
circuit.draw('mpl')
plt.show()

We can determine the probability of the qubit being in the state |1⟩ to demonstrate that the phase information has been successfully encoded onto it.

$$
P[\ket{0}] = |\frac{1+e^{i\theta}}{2}|^2
$$

$$
P[\ket{1}] = |\frac{1-e^{i\theta}}{2}|^2
$$

## Eulers Formula:
$$e^{i\theta} = cos(\theta)+isine(\theta)$$
$$e^{i\theta} + e^{-i\theta} = cos(\theta) + isine(\theta) +  cos(\theta) - isine(\theta) = 2cos(\theta)$$
$$cos(\theta) = \frac{1}{2}\{e^{i\theta} + e^{-i\theta}\}$$
$$|cos(\theta)|^2 = |\frac{e^{i\theta} + e^{-i\theta}}{2}|^2= |\frac{e^{-i \theta} \{ 1+e^{2i \theta}\}}{2}|^2 =|\frac{ 1+e^{2i \theta}}{2}|^2  $$

Replace $\theta$ by $\frac{\theta}{2}$ to equate the equation to $P\ket1$
        $$|cos(\frac{\theta}{2})|^2 =|\frac{ 1+e^{i \theta}}{2}|^2  $$
    using $cos(2\theta) = 1-2sin^2(\theta)$ at step a we will arrive at 
        $$ |sin(\frac{\theta}{2})|^2 =|\frac{ 1-e^{i \theta}}{2}|^2 $$

    

 Extracting the Phase Shift: $P[\ket1] =  |sin(\frac{\theta}{2})|^2 $ and $P[\ket0]= |cos(\frac{\theta}{2})|^2$


# Solving for $P[\ket0]$
$$ P[\ket0]= |cos(\frac{\theta}{2})|^2 = |cos^2(\frac{\theta}{2})| =\frac{1+cos(\theta)}{2} $$
for $\theta \in [0, 2\pi]$


# solving for $P[\ket1]$


$$ sine^2(\theta) + \cos^2(\theta) = 1$$
$$sine^2 = 1 - cos^2(\theta)$$
$$P[\ket1] = 1 - cos^2(\theta) = 1 - \ket1  = \frac{1-cos(\theta)}{2}$$
for $\theta \in [0, 2\pi]$

In [ ]:

# transpile the circuit to the Aer simulator
simulator = Aer.get_backend('qasm_simulator')
compiled_circuit = transpile(circuit, simulator)

# Number of times the experiment is run
shots = 1000

# plot a histogram shots is the number of times the circuit is run(will be elaborated on later)
job = simulator.run(compiled_circuit, shots=shots)
result = job.result()

counts = result.get_counts()

# probbility of the qubit being in the 0 state
probability0 = 0 if '0' not in counts else counts['0']/shots
probability1 = 0 if '1' not in counts else counts['1']/shots

print("Probability of the qubit being in the 0 state: ", probability0)
print("Probability of the qubit being in the 1 state: ", probability1)


NameError: name 'Aer' is not defined

## Relating the Probability of $\ket{1}$ to $\theta$

We can use this function to relate the probability of $\ket{1}$ to $\theta$ and obtain a reading of $\theta$. We then calculate and print the error between our calculated $\theta$ and the expected value of $\theta = \frac{\pi}{2}$.

In [ ]:

# calculate the phase from the probability of ones
phase = math.acos((probability1*2-1)*-1)

print("Phase: ", phase)

# calculate the phase error
phaseError = abs(phase - phaseAmount)
print("Phase Error: ", phaseError)

Probability of the qubit being in the 0 state:  0.525
Probability of the qubit being in the 1 state:  0.475
Phase:  1.5207754699891265
Phase Error:  0.050020856805770064
